In [16]:
for i in range(len(edges1)):
    for j in range(len(edges2)):
        print(edge_intersection(edges1[i], edges2[j]))

None
None
None
(-151.88444676409188, -67.04744258872651)
(-49.85926222935046, -61.91599839615076)
None
(-47.30642132932782, 67.08524220803605)
(11.196480211517805, -30.089068825910932)
None


In [17]:
#Checking for self-intersecting lines
# Cari intersection point, bikin segment, Tentuin segment di dalem apa dilua, trus apa kek tunjukin segmentnya dmn

import turtle as T

T.hideturtle()
Sc = T.Screen()
A = [(-149.0, 236.0), (-310.0, -75.0), (28.0, -58.0)]
B = [(61.0, 206.0), (-215.0, -148.0), (255.0, 97.0)]

def display(polygon):
    T.pu()
    T.goto(polygon[0])
    T.pd()
    for p in polygon[1:]:
        T.goto(p)
    T.goto(polygon[0])  # Close the polygon
    T.update()          # Refresh screen
    
    
display(A)
display(B)
T.done()

In [18]:
def get_edges(polygon):
    """
    Given a polygon as a list of (x, y) tuples, return a list of edges.
    Each edge is ((x1, y1), (x2, y2))
    Each polygon of three vertices (points) has three edges, thus need to wrap from last to first.
    """
    edges = []
    n = len(polygon)
    for i in range(n):
        p1 = polygon[i]
        p2 = polygon[(i + 1) % n]  # wraps last->first
        edges.append((p1, p2))
    return edges

A = [(-149.0, 236.0), (-310.0, -75.0), (28.0, -58.0)]
B = [(61.0, 206.0), (-215.0, -148.0), (255.0, 97.0)]

edges1 = get_edges(A)
edges2 = get_edges(B)

print(edges1)
print(edges2)

[((-149.0, 236.0), (-310.0, -75.0)), ((-310.0, -75.0), (28.0, -58.0)), ((28.0, -58.0), (-149.0, 236.0))]
[((61.0, 206.0), (-215.0, -148.0)), ((-215.0, -148.0), (255.0, 97.0)), ((255.0, 97.0), (61.0, 206.0))]


In [19]:
def edge_intersection(e1, e2, eps=1e-9):
    '''
    Each edge is ((x11, y11), (x12, y12))
    '''
    #Assignment operator
    (x11, y11), (x12, y12) = e1  # Edge 1 endpoints: P11, P12
    (x21, y21), (x22, y22) = e2  # Edge 2 endpoints: P21, P22
    
     # Direction vectors V1 = P12 - P11, V2 = P22 - P21
    v1x, v1y = x12 - x11, y12 - y11
    v2x, v2y = x22 - x21, y22 - y21
    
    denominator = v1x * v2y - v1y * v2x
    
    #Parallel case
    if abs(denominator) < eps: 
        return None
    
    # Numerators from Appendix B (Eqns 5 and 6)
    # A1 = (x11, y11), A2 = (x21, y21)
    t1 = ((x21 * v2y - y21 * v2x) - (x11 * v2y - y11 * v2x)) / denominator
    t2 = ((x21 * v1y - y21 * v1x) - (x11 * v1y - y11 * v1x)) / denominator

    # Check if intersection lies within both segments (0 <= t <= 1)
    if 0 <= t1 <= 1 and 0 <= t2 <= 1:
        ix = x11 + v1x * t1
        iy = y11 + v1y * t1
        return (ix, iy)  # intersection point
    else:
        return None

print(edge_intersection(edges1[0], edges2[0]))

None


In [20]:
def segment_edge(edge, intersection_points):
    '''
    After 
    Given an edge and a list of intersection points on that edge,
    return all sub-segments created by those intersection points.
    
    Edge: ((x1, y1), (x2, y2))
    intersection_points: list of (x, y) tuples that lie on this edge
    
    Returns: list of sub-segments sorted by distance from start point
    '''
    
    (x1, y1), (x2, y2) = edge
    
    # Combined list with all the points
    points_on_edge = [(x1, y1)] + intersection_points + [(x2, y2)]
    
    # Sort points by Euclidean distance from the start point (x1, y1) 
    def distance_from_start(point):
        px, py = point
        return ((px - x1)**2 + (py - y1)**2)**0.5  # sqrt((x2-x1)^2 + (y2-y1)^2)
    
    points_on_edge.sort(key=distance_from_start)
    
    # Create segments between consecutive points
    segments = []
    for i in range(len(points_on_edge) - 1):
        segments.append((points_on_edge[i], points_on_edge[i + 1]))
    
    return segments

# Test it
edge = ((-149.0, 236.0), (-310.0, -75.0))
intersections = [(-200.0, 100.0), (-180.0, 150.0)]
result = segment_edge(edge, intersections)
print(result)

[((-149.0, 236.0), (-180.0, 150.0)), ((-180.0, 150.0), (-200.0, 100.0)), ((-200.0, 100.0), (-310.0, -75.0))]


In [21]:
def polygon_intersections(poly1, poly2):
    #get the edges of both polygons
    edges1 = get_edges(poly1)
    edges2 = get_edges(poly2)
    
    #Segments
    segments_poly1 = []
    segments_poly2 = []

    #intersections
    intersections = []

    #finding all intersections
    for i in range(len(edges1)):
        temp_intersections = []
        for j in range(len(edges2)):
            p = edge_intersection(edges1[i], edges2[j])
            if p is not None:
                temp_intersections.append(p)
        segments_poly1.append(segment_edge(edges1[i], temp_intersections)) #that way it knows which edge its referencing and that the points will always be on that specific edge, stored in temp_intersections
        segments_poly2.append(segment_edge(edges2[j], temp_intersections))
        intersections.extend(temp_intersections) #append so that the segments are stored for each edge.           
    return segments_poly1, segments_poly2, intersections

seg1, seg2, intersections = polygon_intersections(A, B)

print("=== Polygon A Segmented Edges ===")
print(seg1)
print("\n=== Polygon B Segmented Edges ===")
print(seg2)
print("\n=== Intersection Points ===")
print(intersections)




=== Polygon A Segmented Edges ===
[[((-149.0, 236.0), (-310.0, -75.0))], [((-310.0, -75.0), (-151.88444676409188, -67.04744258872651)), ((-151.88444676409188, -67.04744258872651), (-49.85926222935046, -61.91599839615076)), ((-49.85926222935046, -61.91599839615076), (28.0, -58.0))], [((28.0, -58.0), (11.196480211517805, -30.089068825910932)), ((11.196480211517805, -30.089068825910932), (-47.30642132932782, 67.08524220803605)), ((-47.30642132932782, 67.08524220803605), (-149.0, 236.0))]]

=== Polygon B Segmented Edges ===
[[((255.0, 97.0), (61.0, 206.0))], [((255.0, 97.0), (61.0, 206.0)), ((61.0, 206.0), (-49.85926222935046, -61.91599839615076)), ((-49.85926222935046, -61.91599839615076), (-151.88444676409188, -67.04744258872651))], [((255.0, 97.0), (61.0, 206.0)), ((61.0, 206.0), (11.196480211517805, -30.089068825910932)), ((11.196480211517805, -30.089068825910932), (-47.30642132932782, 67.08524220803605))]]

=== Intersection Points ===
[(-151.88444676409188, -67.04744258872651), (-49.8

In [22]:
#After getting intersection, then I should work on the Seeing which segment is inside or outside the other polygon